In [22]:
# -*- coding: utf-8 -*-
import asyncio
import json
import pandas as pd
from api.binance_api import get_signal_performance_data, get_previous_data, API_INTERVALS

from utils.data_utils import extract_price, convert_to_dict_financial_data, get_previous_close_price_for_rsi
from utils.finance_utils import calculate_rsi
from utils.time_utils import convert_time_to_utc_millis

from visualization.plot_utils import visualize_entry_point

In [23]:
file_path = '../signals_data/alert_from_algo.csv'
signals_df = pd.read_csv(file_path)
KLINE_BITCOIN_PUMP = 1

In [24]:
signals_df.rename(columns={signals_df.columns[0]: 'id'}, inplace=True)
signals_df.loc[signals_df['symbol'] == '‼️‼️‼️‼️‼️', 'symbol'] = 'BTCUSDT'
signals_df.loc[signals_df['emoji'] == 'BITCOIN PUMP', 'emoji'] = '🌊🌊🌊'
signals_df.loc[signals_df['kline'] == '‼️‼️‼️‼️‼️', 'kline'] = f'{KLINE_BITCOIN_PUMP} kline'
signals_df['kline'] = signals_df['kline'].str.extract('(\d+)').astype(int)
signals_df['alert_price'] = signals_df['alert_price'].str.extract(r'(\d+\.\d+)').astype(float)

In [31]:
signals_df

,id,symbol,emoji,kline,alert_price,Time_(MSK)
0,0,ANTUSDT,💥💥💥,1,5.20100,2023-11-09 18:51:52.578000
1,1,ANTUSDT,💥💥💥,1,5.20100,2023-11-09 18:51:52.578000
2,2,PENDLEUSDT,💥💥💥,1,1.04230,2023-11-09 18:50:35.789000
3,3,PENDLEUSDT,💥💥💥,1,1.04230,2023-11-09 18:50:35.789000
4,4,YFIUSDT,💥💥💥,1,6868.00000,2023-11-09 18:48:24.505000
...,...,...,...,...,...,...
96,96,AUDIOUSDT,💥💥💥,1,0.19150,2023-11-09 17:09:44.845000
97,97,AUDIOUSDT,💥💥💥,1,0.19150,2023-11-09 17:09:44.845000
98,98,SKLUSDT,🌊🌊🌊,2,0.03009,2023-11-09 17:01:41.917000
99,99,GMXUSDT,🌊🌊🌊,2,50.42000,2023-11-09 17:01:33.226000


In [33]:
signals_df.dtypes

id               int64
symbol          object
emoji           object
kline            int32
alert_price    float64
Time_(MSK)      object
dtype: object

In [39]:
async def fetch_signal_data(row):
    # Определение, является ли сигнал сигналом на повышение или понижение на основе эмодзи
    if row['emoji'] == '💥💥💥':
        rise=False # Сигнал на понижение
    elif row['emoji'] == '🌊🌊🌊':
        rise=True # Сигнал на повышение

    # Извлечение данных о символе, цене оповещения и времени из строки DataFrame
    symbol = row['symbol']
    alert_price = row['alert_price']
    clear_time = convert_time_to_utc_millis(row['Time_(MSK)'])

    # Получение данных об эффективности сигнала
    signal_data = await get_signal_performance_data(symbol, alert_price, clear_time, interval='1m', n_intervals=5, deviation_threshold=3, check_rise=rise)
    signal_data = json.loads(signal_data)

    return row['id'], signal_data['status'], signal_data['price_change']

async def process_signals_df(signals_df):
    # Создание асинхронных задач для обработки каждой строки DataFrame
    tasks = [fetch_signal_data(row) for _, row in signals_df.iterrows()]
    results = await asyncio.gather(*tasks) # Асинхронный запуск всех задач

    # Преобразование результатов в DataFrame
    results_df = pd.DataFrame(results, columns=['id', 'status', 'price_change'])
    return results_df

# Вызов асинхронной функции в Jupyter Notebook
results_df = await process_signals_df(signals_df)

# Объединение результатов с исходным DataFrame
final_df = pd.merge(signals_df, results_df, on='id')


In [40]:
final_df

,id,symbol,emoji,kline,alert_price,Time_(MSK),status,price_change
0,0,ANTUSDT,💥💥💥,1,5.20100,2023-11-09 18:51:52.578000,Failure,0.269179
1,1,ANTUSDT,💥💥💥,1,5.20100,2023-11-09 18:51:52.578000,Failure,0.269179
2,2,PENDLEUSDT,💥💥💥,1,1.04230,2023-11-09 18:50:35.789000,Failure,0.892258
3,3,PENDLEUSDT,💥💥💥,1,1.04230,2023-11-09 18:50:35.789000,Failure,0.892258
4,4,YFIUSDT,💥💥💥,1,6868.00000,2023-11-09 18:48:24.505000,Failure,0.291206
...,...,...,...,...,...,...,...,...
96,96,AUDIOUSDT,💥💥💥,1,0.19150,2023-11-09 17:09:44.845000,Failure,0.731070
97,97,AUDIOUSDT,💥💥💥,1,0.19150,2023-11-09 17:09:44.845000,Failure,0.731070
98,98,SKLUSDT,🌊🌊🌊,2,0.03009,2023-11-09 17:01:41.917000,alert_price not in low-high range,NaN
99,99,GMXUSDT,🌊🌊🌊,2,50.42000,2023-11-09 17:01:33.226000,alert_price not in low-high range,NaN


In [41]:
status_counts = final_df.groupby('status').count()
status_counts

,id,symbol,emoji,kline,alert_price,Time_(MSK),price_change
status,,,,,,,
Failure,56,56,56,56,56,56,56
alert_price not in low-high range,45,45,45,45,45,45,0


In [42]:
success_count = status_counts.loc['Success', 'kline']
f'Количество успешных сигналов: {success_count}'

KeyError: 'Success'

In [30]:
async def calculate_rsi_for_row(symbol, time, interval='1m', period=14):
    start_time = convert_time_to_utc_millis(time)
    historical_data = await get_previous_data(symbol, start_time, interval, period)
    close_prices_list = [item['Close'] for item in historical_data]
    rsi_values = json.loads(calculate_rsi(close_prices_list))
    return rsi_values['rsi_list'][-1]  # Returns the latest RSI value

async def append_rsi_to_dataframe(df):
    rsi_data = []
    for _, row in df.iterrows():
        rsi_value = await calculate_rsi_for_row(row['symbol'], row['Time_(MSK)'])
        rsi_data.append({'id': row['id'], 'RSI': rsi_value})

    rsi_df = pd.DataFrame(rsi_data)
    merged_df = pd.merge(df, rsi_df, on='id')
    return merged_df

# Calculate and append RSI values
final_df_with_rsi = await append_rsi_to_dataframe(final_df)

In [34]:
final_df_with_rsi

,id,symbol,emoji,kline,alert_price,Time_(MSK),status,price_change,RSI
0,0,ANTUSDT,💥💥💥,1,5.20100,2023-11-09 18:51:52.578000,Failure,-0.711402,66.842105
1,1,ANTUSDT,💥💥💥,1,5.20100,2023-11-09 18:51:52.578000,Failure,-0.711402,66.842105
2,2,PENDLEUSDT,💥💥💥,1,1.04230,2023-11-09 18:50:35.789000,Failure,-0.316608,70.331950
3,3,PENDLEUSDT,💥💥💥,1,1.04230,2023-11-09 18:50:35.789000,Failure,-0.316608,70.331950
4,4,YFIUSDT,💥💥💥,1,6868.00000,2023-11-09 18:48:24.505000,Failure,-0.436808,60.765550
...,...,...,...,...,...,...,...,...,...
96,96,AUDIOUSDT,💥💥💥,1,0.19150,2023-11-09 17:09:44.845000,Failure,0.000000,76.470588
97,97,AUDIOUSDT,💥💥💥,1,0.19150,2023-11-09 17:09:44.845000,Failure,0.000000,76.470588
98,98,SKLUSDT,🌊🌊🌊,2,0.03009,2023-11-09 17:01:41.917000,alert_price not in low-high range,NaN,90.697674
99,99,GMXUSDT,🌊🌊🌊,2,50.42000,2023-11-09 17:01:33.226000,alert_price not in low-high range,NaN,77.551020
